ugh I'm getting bad at naming new functions, but this is going to be almost exactly the same as the array version of the solver, but now I'm going to work through it in a parallel fashion, it'll be able to take in one det val array, and return chi squared, but this time it'll be able to say if one chi term is supposed to be bad a la the conditions I had early on. 

so...

input:  detval, detnorm, thetarange,phirange, Arange?, n, bg... same as usual

returns: chisquared array of this specific detector, these are all to be added, secret here is to use the mask to turn the chiterms that are supposed to be bad, bad. 

In [ ]:
def speedy_solver(detsvals,detnorms,bottheta,toptheta,botphi,topphi,background):
   # mask2 = np.array(detsvals) >background
    #detsvals = detsvals[mask2]
    theta = np.deg2rad(np.linspace(bottheta,toptheta,3))
    phi = np.deg2rad(np.linspace(botphi,topphi,4))
    mphi,mtheta = np.meshgrid(phi,theta)
    allthetas = np.concatenate(mtheta)
    allphis = np.concatenate(mphi)
    allvecs = hp.ang2vec(allthetas,allphis)
    As= np.linspace(10,1000,5)
    chiterms = np.zeros(len(As)*len(theta)*len(phi))
   # print("Len chi terms: the zeros one.. " + str(len(chiterms)))
    for s in range(len(detsvals)):
        normarr = detnorms[s]  #an array that is the normal vector for a given detector, changes for each loop. 
   #    print("normal array " + str(s) + " " + str(np.rad2deg(hp.vec2ang(normarr)))) 
        normarrs = []
        for garc in range((len(theta)*len(phi))):
            normarrs.append([normarr[0],normarr[1],normarr[2]])
        
        seps = findAngles(allvecs,normarrs)
        
        #mask = seps < np.pi/2
        #seps[mask]
        #only using seps under 90, good. 
        AA,SS = np.meshgrid(As,seps)

        Aofit = np.concatenate(AA)
        chiseps = np.concatenate(SS)
        bg = background * np.ones(len(chiseps))
        #when chiseps in here are >90, make response huuuge
        good = chiseps < np.pi/2
        bad = chiseps > np.pi/2
        chiResponse = np.multiply(Aofit,response(chiseps)) + bg
         
    #  print("iteration : " + str(s))
        chiterm = np.divide(np.power(np.subtract(chiResponse,detsvals[s]),2),detsvals[s])
      #  print("chiterm: " + str(chiterm))

        chiterms += chiterm
     #   print("chiterms final: " + str(chiterms))
    chimin = min(chiterms)
    chisquareds = list(chiterms)
   # thetaloc = np.rad2deg(theta[int((chisquareds.index(chimin)-(chisquareds.index(chimin) % (len(phi)*len(Aofit))))/(len(phi)*len(Aofit)))])
   # philoc = np.rad2deg(phi[int(((chisquareds.index(chimin) % (len(phi)*len(Aofit)))-(chisquareds.index(chimin) % (len(Aofit))))/len(Aofit))])
  #  Aoguess=Aofit[int((chisquareds.index(chimin) % (len(phi)*len(Aofit)))  % len(Aofit))]

    return  chiterm

In [ ]:
def quad_solver(detval,detnorm,bottheta,toptheta,botphi,topphi,background):
    theta = np.deg2rad(np.linspace(bottheta,toptheta,3))
    phi = np.deg2rad(np.linspace(botphi,topphi,4))
    mphi,mtheta = np.meshgrid(phi,theta)
    allthetas = np.concatenate(mtheta)
    allphis = np.concatenate(mphi)
    allvecs = hp.ang2vec(allthetas,allphis)
    As= np.linspace(10,1000,5)
    normarr = detnorm
    normarrs = []
    for garc in range((len(theta)*len(phi))):
        normarrs.append([normarr[0],normarr[1],normarr[2]])
        
    seps = findAngles(allvecs,normarrs)
    AA,SS = np.meshgrid(As,seps)
    Aofit = np.concatenate(AA)
    chiseps = np.concatenate(SS)
    [np.pi/2 if x>np.pi/2 else x for x in chiseps]

    #this is close, but needs to be a way to go one step further and do this in the next chiR
    bg = background * np.ones(len(chiseps))
    good = chiseps < np.pi/2
    bad = chiseps > np.pi/2
    
    
    chiResponse = np.multiply(Aofit,response(chiseps)) + bg

    if detval > background: 
        chiterm = np.divide(np.power(np.subtract(chiResponse,detval),2),detval)
    else: 
        chiterm = 1e5 * np.ones(len(theta)*len(phi)*len(As))
    
    return chiterm

In [3]:
np.add([0,1],  [1,10])

array([ 1, 11])

In [2]:
import numpy as np

In [3]:
a=[1,2,3,1,3,2,1,1]
[4 if x<=2 else x for x in a]


[4, 4, 3, 4, 3, 4, 4, 4]